### 1.d. - Autitoria no Logadouro / Nome das ruas (Street)

Para identificação de tipos de Logradouros / nomes de ruas tipo é utilizada a seguinte regra:

- Tags "way" e subtag "tag" atributo "name", onde para este mesmo tag "way" também exista um subtag chamado "tag" com atributo "k" = "highway"
<br/>Ou<br;> 
- tag "node" e subtag "tag" e atributo "add:street"

Para o primeiro caso, existem uma infinidade de elementos que não são ruas. O código abaixo utiliza o atributo "highway" com a finalidade de identitificar o que é logradouro / rua.

Foram levantados os tipos de logradouro/ruas mais comuns e definiros na variável "expected". O que estiver fora destes valores, serão considerados inconsistentes e deverão ser analisados.

Os nomes de ruas com caracteres especiais também serão considerados como inconsistêntes.

In [1]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

# Arquivo XML que a ser auditado
OSMFILE = "dados/jundiai_e_regiao_map_zen.osm"

# Variáveis com padrões de expressões regulares
accept = re.compile(r'^\w*$')
upper = re.compile(r'^([A-Z])*$')
lower_and_upper = re.compile(r'^([A-z]| )*$')
problemchars = re.compile(r'[=\+/&<>;"\?%#$@\,\t\r\n]')
with_number = re.compile(r'^[0-9]')


# O que for diferente destes tipos, devem obrigatoriamente ser exibidos
expected = ["Rua", "Avenida", "Rodovia", "Travessa", "Viaduto", "Alameda", "Praça", "Via", "Estrada",
            "Viela", "Calçadão", "Largo", "Rodoanel", "Rotatória", "Alameda", "Caminho", "Corredor",
            "Passagem", "Acesso", "Marginal", "Trevo", "Passarela", "Contorno", "Travessia",
            "Paço", "Entrada", "Túnel", "Entrada", "Travessia", "Saída", "Alamedas", "Vila", "Trilha"]


def audit_street_type(street_name):
    """ Tipo de logradouro/rua é esperado ?
    
    Verifica se o tipo de logradouro/rua esta entre os valores esperados
    
    Argumentos:
        street_types (str): Nome do logradouro/rua
        
    Retorno:
        True se o tipo do logradouro/rua estiver dentro dos tipos esperados (variável expected). False para os 
        outros casos. 
    """
    result = False
    
    street_type = street_name.split()[0]
    if street_type not in expected:
        result = False
    else:
        result = True

    return result


def audit_characters_street(element):
    """ Valida caracteres do logradouro/rua
    
    Através de Expressões regulares, verificar se o logradouro/rua tem alguma irregularidade.
    
    Argumentos:
        element (str): Nome do logradouro/rua
    
    Retorno:
        False se for encontrada alguma irregularidade e True para os outros casos.
    """
    
    result = False
    classificated = False
    
    # UPPER
    if upper.search(element) is not None:
        classificated = True
    
    # LOWER
    if not classificated:    
        if accept.search(element) is not None:
            result = True
            classificated = True
            classificated = True        
    
    # LOWER AND UPPER
    if not classificated:
        if lower_and_upper.search(element) is not None:
            result = True
            classificated = True        
            
    # with_number
    if not classificated:
        if with_number.search(element) is not None:        
            result = True
            classificated = True        

    # PROBLEMCHAR
    if problemchars.search(element) is not None:
        classificated = True        
        
    # OUTROS
    if not classificated:
        result = True
    
    return result


def audit(osmfile):
    """ Auditoria no logardouro / nomes das ruas
    
    Realizar varredura no arquivo XML em busca de logradouros / nomes de ruas. Após identificados, realizar
    validação e exibir em tela os casos onde forem identificados caracteres especiais ou tipos de logardouros 
    diferente dos esperados.
    
    Argumentos:
        osmfile (str) : Caminho e nome do arquivo a ser auditado
    
    """
    
    osm_file = open(osmfile, "r", encoding="utf-8")
    
    street_types = defaultdict(set)
    id_tag_highway = []
    
    for event, elem in ET.iterparse(osm_file, events=("start",)):
                
        if elem.tag == "node" or elem.tag == "way":
     
            # Identificar tags way que contém atributo k = "highway"  
            # Se existir um atributo "k" com valor "highway", será adicionado na variável id_tag_highway 
            # para ser utilizado posteriormente na identificação do que é logadouro/rua
            if elem.tag == "way":
                for tag in elem.iter("tag"):
                    if tag.attrib['k'] == "highway":
                        if elem.attrib["id"] not in id_tag_highway:
                            id_tag_highway.append(elem.attrib["id"])
                        break
            
            for tag in elem.iter("tag"):
                # Validação do tag way
                if (elem.tag == "way" and tag.attrib['k'] == "name"):
                    
                    # Verifica se contem a subtag highway (identificado anteriormente)
                    if elem.attrib["id"] in id_tag_highway:
                        
                        # Se estiver entre os tipos de logradouro/ruas esperados, validar os caracteres. Caso 
                        # Contrário, exibir em tela para verificação 
                        if audit_street_type(street_types, tag.attrib['v']):
                            if audit_characters_street(tag.attrib['v']) == False:
                                print(tag.attrib)                                
                        else:
                            print(tag.attrib)
                            
                # Validação do tag node
                elif (elem.tag == "node" and tag.attrib['k'] == "addr:street"):
                    
                    # Se estiver entre os tipos de ruas esperados, validar os caracteres. Caso contrário, 
                    # exibir na tela para verificação
                    if audit_street_type(street_types, tag.attrib['v']):
                        if audit_characters_street(tag.attrib['v']) == False:
                            print(tag.attrib)
                    else:
                        print(tag.attrib)
                
    osm_file.close()


def test():
    # Auditar para encontrar nome das ruas fora do padrão
    audit(OSMFILE)      
            

if __name__ == '__main__':
    test()


{'k': 'addr:street', 'v': 'estrada Tahira Eki'}
{'k': 'name', 'v': 'Rua " 30 "'}
{'k': 'name', 'v': 'Rua " 26 "'}
{'k': 'name', 'v': 'Rua "1"'}
{'k': 'name', 'v': 'Rua "2"'}
{'k': 'name', 'v': 'Complexo Viário Tobias Muzaiel'}
{'k': 'name', 'v': 'Rua "04"'}
{'k': 'name', 'v': 'Avenida dos Alpes; Avenida Pauliceia'}
{'k': 'name', 'v': 'Avenidas das Aves Marinhas'}
{'k': 'name', 'v': 'Rau Diogo Alveres correia'}
{'k': 'name', 'v': 'rua Raul Breesane Malta'}
{'k': 'name', 'v': 'Miguel De Barros'}
{'k': 'name', 'v': 'José Pereira da Silva'}
{'k': 'name', 'v': 'Rod. Manoel Silvério Pinto'}
{'k': 'name', 'v': 'Rua Vitoria,'}
{'k': 'name', 'v': 'Rua "R-5"'}
{'k': 'name', 'v': 'José Vitórino Ferreira Filho'}
{'k': 'name', 'v': 'Rua Manoel Pinto Rodrigues,'}
{'k': 'name', 'v': 'Rua " 16 "'}
{'k': 'name', 'v': 'Pedro Barsanelli'}
{'k': 'name', 'v': 'Antiga Estrada da Mata Fria'}
{'k': 'name', 'v': 'Ponte Velha'}
{'k': 'name', 'v': 'Doutor Osvaldo Urioste'}
{'k': 'name', 'v': 'Ru Fernão Dias'}
{'